In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn

# spacy for lemmatization
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import nltk

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\joblib\backports.py:22: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils  # noqa
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprec

In [3]:
df = pd.read_csv('./dataset/NMSendataset_wordtoken_lda_2023.csv', error_bad_lines=False);
df

,_id,recommend,found_helpful,found_funny,Column1,time,reply_count,content,hrs_on_second,hrs_at_reviwer_time,reviwer_num,language,sent_content,preprocessed_reviews_sec,wordtoken,correctmapping,preprocessed_reviews_lda
0,5f75ad269136b25c3c8b50f8,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tthis good\t\t\t\t\t\t\t,0.5,37.9,15,en,\r\n\t\t\t\t\t\t\t\tthis good,this good,['good'],"['this', 'good']",good
1,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,what a difference a few year make .,"['difference', 'year']","['what', 'difference', 'few', 'year', 'make']",difference year make
2,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,"From its messy origins, No Man's Sky has grown...",from it messy origin: no man be sky have grown...,"['messy', 'origin', 'no_man_sky', 'grown', 'de...","['from', 'it', 'messy', 'origin', 'no', 'man',...",messy origin no man sky grown deeper polished ...
3,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,"The vast universe is enticing, and there's ple...",the vast universe be entice: and there be plen...,"['vast', 'universe', 'entice', 'plenty', 'most...","['the', 'vast', 'universe', 'be', 'entice', 'a...",vast universe entice plenty mostly either expl...
4,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,"And with proper multiplayer now introduced, it...",and with proper multiplayer now introduce: it ...,"['proper', 'multiplayer', 'introduce', 'lonely...","['and', 'with', 'proper', 'multiplayer', 'now'...",proper multiplayer introduce lonely experience...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476338,64abdb8a38d28815033e7a39,False,0,0,5,12/08/2016,1,\r\n\t\t\t\t\t\t\t\tThis isn't a PC game. It'...,0.0,44.9,37,en,"If the game was $20 I'd keep it, but definitel...",if the game be $ 20 i would keep it: but defin...,"['20', 'would', 'definitely', 'worth_60', 'ref...","['if', 'the', 'game', 'be', '20', 'would', 'ke...",20 would keep definitely worth 60 refund time
476339,64abf8e1b4d8ae53485d2c42,True,0,0,5,12/08/2016,0,\r\n\t\t\t\t\t\t\t\tIt's pretty interesting\t\...,0.0,23.2,42,en,\r\n\t\t\t\t\t\t\t\tIt's pretty interesting,\r\n it be pretty interest,"['pretty', 'interest']","['it', 'be', 'pretty', 'interest']",pretty interest
476340,64ac11efb4d8ae53485d2c6b,False,0,0,5,12/08/2016,2,\r\n\t\t\t\t\t\t\t\tI dislike the fact that al...,0.0,53.4,17,en,\r\n\t\t\t\t\t\t\t\tI dislike the fact that al...,\r\n i dislike the fact that all the creature ...,"['dislike', 'fact', 'creature', 'promise', 'cr...","['dislike', 'the', 'fact', 'that', 'all', 'the...",dislike fact creature promise creature keep sh...
476341,64ac11efb4d8ae53485d2c6b,False,0,0,5,12/08/2016,2,\r\n\t\t\t\t\t\t\t\tI dislike the fact that al...,0.0,53.4,17,en,Please add the creatures into the game.,please add the creature into the game .,"['please', 'add', 'creature']","['please', 'add', 'the', 'creature', 'into', '...",please add creature


In [4]:
eval(df['wordtoken'][4])

['proper', 'multiplayer', 'introduce', 'lonely', 'experience', 'either']

In [5]:
def convert(s):
    s = eval(s)
    s = ' '.join(s)
    return s
df['wordtoken'] = df['wordtoken'].apply(convert)

In [6]:
data = df.wordtoken.values.tolist()
data[:3]

['good',
 'difference year',
 'messy origin no_man_sky grown deeper polished developer work something like initial promise']

# Vectorizer

## TF-IDF vectorizer

In [7]:
tfidf_vectorizer = TfidfVectorizer(
                                 max_df=500, 
                                   max_features=2000,
                                 min_df=0, 
#                                  stop_words='english',
                                   use_idf=True, 
#                                    token_pattern='[a-zA-Z0-9]{3,}', 
#                                    ngram_range=(1,3)
)

%time tfidf_vectorized = tfidf_vectorizer.fit_transform(data)

print(tfidf_vectorized.shape)

CPU times: total: 3.48 s
Wall time: 3.58 s
(476343, 2000)


In [8]:
tfidf_data_vectorized = tfidf_vectorizer.fit_transform(data)

In [9]:
tfidf_vectorizer.get_feature_names()

['08',
 '1000',
 '1060',
 '1070',
 '1080',
 '10th',
 '120',
 '14',
 '150',
 '17',
 '180',
 '19',
 '1st',
 '200',
 '2000',
 '2014',
 '2017',
 '2019',
 '2021',
 '2022',
 '2023',
 '21',
 '22',
 '23',
 '250',
 '26',
 '27',
 '28',
 '29',
 '2d',
 '2nd',
 '300',
 '32',
 '35',
 '3d',
 '3rd',
 '400',
 '45',
 '48',
 '480',
 '4gl',
 '4h',
 '4th',
 '500',
 '55',
 '5999',
 '64',
 '65',
 '66',
 '75',
 '85',
 '90s',
 '95',
 '960',
 '980',
 '99',
 'aa',
 'absurd',
 'abundant',
 'abysmal',
 'abyss',
 'accept',
 'acceptable',
 'accessible',
 'accidentally',
 'accomplish',
 'accomplishment',
 'accord',
 'account',
 'accurate',
 'acid',
 'acknowledge',
 'acquire',
 'act',
 'activate',
 'active',
 'actively',
 'activity',
 'additional',
 'additionally',
 'adjust',
 'admire',
 'admittedly',
 'adult',
 'adults',
 'advance',
 'advanced',
 'advantage',
 'advertisement',
 'advice',
 'advise',
 'aesthetic',
 'af',
 'affect',
 'afford',
 'afraid',
 'afterwards',
 'aggressive',
 'aid',
 'aim',
 'aimless',
 'aimles

In [10]:
tfidf_data_feature_names=tfidf_vectorizer.get_feature_names()

In [11]:
tfidf_data_dense = tfidf_data_vectorized.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("tfidf_Sparsicity: ", ((tfidf_data_dense > 0).sum()/tfidf_data_dense.size)*100, "%")
# Since most cells in this matrix will be zero, I am interested in knowing what percentage of cells contain non-zero values.

tfidf_Sparsicity:  0.045371612472525046 %


# Build LDA Model

In [12]:
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(tfidf_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [13]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)

In [14]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(tfidf_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(tfidf_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2485819.485829093
Perplexity:  4597.064920342282
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [15]:
# Define Search Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 'max_iter': [5, 10, 15, 20, 30, 40, 50]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(tfidf_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'max_iter': [5, 10, 15, 20, 30, 40, 50],
                         'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45,
                                          50]})

In [16]:
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
        n_jobs=1,
       param_grid={'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 'max_iter': [5, 10, 15, 20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'max_iter': [5, 10, 15, 20, 30, 40, 50],
                         'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45,
                                          50]},
             return_train_score='warn')

In [17]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(tfidf_vectorized))

Best Model's Params:  {'max_iter': 50, 'n_components': 5}
Best Log Likelihood Score:  -477782.8021196572
Model Perplexity:  2613.666859891924
